# 02 - Data Preparation

Limpieza, imputación, transformaciones iniciales y guardado de datasets listos para EDA, modelado y predicciones.


In [ ]:
# =============================================================================
# CONFIGURACIÓN INICIAL Y DEFINICIÓN DE RUTAS
# =============================================================================
# Importación de bibliotecas necesarias para preparación de datos
import pandas as pd  # Manipulación de datos tabulares
import numpy as np   # Operaciones numéricas
from pathlib import Path  # Manejo de rutas de archivos
from sklearn.model_selection import train_test_split  # División de datos
import joblib  # Serialización de objetos para ML

# Configuración de semilla aleatoria para reproducibilidad
RANDOM_SEED = 42

# Definición de directorios y rutas de archivos
DATA_DIR = Path("../data")
TRAIN_CSV = DATA_DIR / "raw" / "train.csv"  # Dataset de entrenamiento raw
TEST_CSV = DATA_DIR / "raw" / "test.csv"    # Dataset de prueba raw
PROCESSED_DIR = DATA_DIR / "processed"      # Directorio para datos procesados

# Crear directorio de datos procesados si no existe
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

print('Processing...')

In [ ]:
# =============================================================================
# CARGA DE DATOS RAW
# =============================================================================
# Cargar datasets originales desde archivos CSV
train = pd.read_csv(TRAIN_CSV)  # Conjunto de entrenamiento con variable objetivo
test = pd.read_csv(TEST_CSV)    # Conjunto de prueba sin variable objetivo

# Verificar dimensiones iniciales de los datasets
print('Initial shapes — train:', train.shape, 'test:', test.shape)

In [ ]:
# =============================================================================
# LIMPIEZA DE DATOS: DUPLICADOS Y TIPOS DE DATOS
# =============================================================================

# 1. Eliminación de registros duplicados basados en ID
# Mantener solo la primera ocurrencia de cada ID único
train = train.drop_duplicates(subset=['id'])
test = test.drop_duplicates(subset=['id'])

# 2. Conversión de variable categórica a tipo 'category' para eficiencia de memoria
# La variable 'Sex' es categórica con solo 2 valores posibles
if 'Sex' in train.columns and train['Sex'].dtype != 'category':
    train['Sex'] = train['Sex'].astype('category')
if 'Sex' in test.columns and test['Sex'].dtype != 'category':
    test['Sex'] = test['Sex'].astype('category')

# 3. Asegurar tipos numéricos correctos para todas las variables numéricas
# Convertir a numérico y manejar errores convirtiéndolos a NaN
numeric_cols_train = ['Age','Height','Weight','Duration','Heart_Rate','Body_Temp','Calories']
for c in numeric_cols_train:
    if c in train.columns:
        train[c] = pd.to_numeric(train[c], errors='coerce')

numeric_cols_test = ['Age','Height','Weight','Duration','Heart_Rate','Body_Temp']
for c in numeric_cols_test:
    if c in test.columns:
        test[c] = pd.to_numeric(test[c], errors='coerce')

# Verificar valores nulos resultantes de la conversión
print('Nulos en train:\n', train.isna().sum())
print('Nulos en test:\n', test.isna().sum())

In [ ]:
# =============================================================================
# IMPUTACIÓN DE VALORES FALTANTES
# =============================================================================
# Estrategia: Imputar con mediana (numéricas) o moda (categóricas) si % nulos < 20%
# Si % nulos >= 20%, se considera un problema serio que requiere análisis adicional

# Definir umbral de aceptación de valores nulos (20% del tamaño del dataset)
threshold = 0.2 * len(train)

# Imputación en dataset de entrenamiento
for col in train.columns:
    miss = train[col].isna().sum()
    if miss > 0:
        if miss < threshold:
            # Imputar variables numéricas con mediana (robusto a outliers)
            if col in numeric_cols_train:
                train[col].fillna(train[col].median(), inplace=True)
            else:
                # Imputar variables categóricas con moda (valor más frecuente)
                train[col].fillna(train[col].mode().iloc[0], inplace=True)
        else:
            # Advertir si hay más del 20% de valores nulos
            print(f"WARNING: {col} tiene {miss} nulos (>20%) en train")

# Imputación en dataset de prueba (solo para columnas existentes)
for col in numeric_cols_test:
    if col in test.columns:
        miss = test[col].isna().sum()
        if miss > 0 and miss < threshold:
            # Usar mediana del test set (no del train para evitar data leakage en producción)
            test[col].fillna(test[col].median(), inplace=True)

# Verificar que la imputación fue exitosa
print('Nulos post-imputacion (train):\n', train.isna().sum())
print('Nulos post-imputacion (test):\n', test.isna().sum())

In [ ]:
# =============================================================================
# FILTRADO DE VALORES FUERA DE RANGO
# =============================================================================
# Aplicar reglas de negocio para eliminar registros con valores no realistas
# Estos rangos están basados en límites fisiológicos razonables

# Filtros aplicados al conjunto de entrenamiento:
# - Edad entre 10 y 100 años (rango humano razonable)
# - Temperatura corporal entre 30°C y 43°C (límites de supervivencia)
# - Duración mayor a 0 (no tiene sentido ejercicio de 0 minutos)
train = train[(train['Age'] >= 10) & (train['Age'] <= 100)]
train = train[(train['Body_Temp'] >= 30) & (train['Body_Temp'] <= 43)]
train = train[train['Duration'] > 0]

# Aplicar los mismos filtros al conjunto de prueba
# Verificar existencia de columnas antes de filtrar (test no tiene Calories)
if 'Age' in test.columns:
    test = test[(test['Age'] >= 10) & (test['Age'] <= 100)]
if 'Body_Temp' in test.columns:
    test = test[(test['Body_Temp'] >= 30) & (test['Body_Temp'] <= 43)]
if 'Duration' in test.columns:
    test = test[test['Duration'] > 0]

# Verificar dimensiones después del filtrado
print('Shapes post filter — train:', train.shape, 'test:', test.shape)

In [ ]:
# =============================================================================
# CODIFICACIÓN DE VARIABLES CATEGÓRICAS Y DIVISIÓN DE DATOS
# =============================================================================

# 1. One-Hot Encoding de la variable 'Sex'
# drop_first=True evita la trampa de variables dummy (multicolinealidad perfecta)
# Mantiene solo 'Sex_male' (1 si male, 0 si female)
if 'Sex' in train.columns:
    train = pd.get_dummies(train, columns=['Sex'], drop_first=True)
    # Convertir booleano a entero para compatibilidad con modelos
    train['Sex_male'] = train['Sex_male'].astype(int)
    
if 'Sex' in test.columns:
    test = pd.get_dummies(test, columns=['Sex'], drop_first=True)
    test['Sex_male'] = test['Sex_male'].astype(int)

# 2. Alinear columnas entre train y test
# Asegurar que test tenga todas las columnas dummy creadas
for col in ['Sex_male']:
    if col not in test.columns:
        test[col] = 0  # Valor por defecto si la columna no existe

# 3. División del conjunto de entrenamiento en train y validación
# 80% entrenamiento, 20% validación
# random_state asegura reproducibilidad de la división
train_proc, val_proc = train_test_split(
    train, 
    test_size=0.2,  # 20% para validación
    random_state=RANDOM_SEED
)

# 4. Guardar datasets procesados para uso en notebooks posteriores
PROCESSED_DIR = Path("../data/processed")
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

train_proc.to_csv(PROCESSED_DIR / 'train.csv', index=False)  # Entrenamiento (80%)
val_proc.to_csv(PROCESSED_DIR / 'val.csv', index=False)      # Validación (20%)
test.to_csv(PROCESSED_DIR / 'test.csv', index=False)         # Prueba (sin Calories)

print('Saved processed datasets in', PROCESSED_DIR)

Siguiente: realizar un análisis exploratorio en `03_exploratory_analysis.ipynb` usando train y val (test se reserva para predicciones porque no tiene `Calories`).
